In [3]:
# !python --version

Python 3.10.9


In [1]:
import langchain
import chromadb

In [6]:
!pip install langchain==0.0.267 chromadb==0.4.6

     -------------------------------------- 405.5/405.5 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 58.4/58.4 kB 763.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.2-cp310-cp310-win_amd64.whl size=150456 sha256=21119b64eed1c1b72b28f0965d49c69393f7400d0992fa6f3f6ea3711a78fd0e
  Stored in directory: c:\users\abhay\appdata\local\pip\cache\wheels\33\94\94\276cab4fe8e1ae164bbfab7560e9ca6cedc7296ee3bd9607ec
Successfully built chroma-hnswlib
  Attempting uninstall: chroma-hnswlib
    Found existing installation: chroma-hnswlib 0.7.3
    Uninstalling chroma-hnswlib-0.7.3:
      Successfully uninstalled chroma-h

In [3]:
print(langchain.__version__,chromadb.__version__)

0.0.267 0.4.6


In [8]:
!pip install -r ../src/requirements.txt

Ignoring protobuf: markers 'sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring protobuf: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring bitsandbytes: markers 'sys_platform != "win32"' don't match your environment
     ------------------------------------ 246.5/246.5 kB 688.9 kB/s eta 0:00:00
     -------------------------------------- 904.0/904.0 kB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 52.7/52.7 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.7/1.7 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 138.5/138.5 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 2.5 MB/s eta 0:00:00
     ------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\abhay\\Documents\\anaconda3\\Lib\\site-packages\\~-rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [ ]:
# from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

# import click

In [11]:
import logging
import os

import torch
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader

In [18]:
print(os.getcwd())
print(os.path.dirname(os.path.realpath(__file__)))

C:\Users\abhay\OneDrive\Desktop\quantify\repo\quantiphi_QA_pdf


NameError: name '__file__' is not defined

In [12]:
os.chdir("C:\\Users\\abhay\\OneDrive\\Desktop\\quantify\\repo\\quantiphi_QA_pdf")
print(os.getcwd())
print(os.path.dirname())

C:\Users\abhay\OneDrive\Desktop\quantify\repo\quantiphi_QA_pdf


In [16]:
import constants
from constants import CHUNK_SIZE,CHUNK_OVERLAP, PARENT_CHUNK_SIZE, CHILD_CHUNK_SIZE,PERSIST_DIRECTORY,EMBEDDING_MODEL_NAME,CHROMA_SETTINGS

In [5]:
print(constants.CHUNK_SIZE, PERSIST_DIRECTORY)

1024 ../VECTOR_DBS


In [ ]:
pdf_file = "C:\Users\abhay\OneDrive\Desktop\quantify\repo\quantiphi_QA_pdf\notebooks/SOURCE_DATA_DB/ConceptsofBiology-WEB_Chapter 1-2.pdf"
loader = PyMuPDFLoader(pdf_file)
pages = loader.load()
print(pages[0].metadata)
len(pages)

In [6]:
def load_pdf(pdf_file):
    loader = PyMuPDFLoader(pdf_file)
    pages = loader.load()
#     if len(pages)>0:
    print("Loaded {} of pages from the pdf file".format(len(pages)))
    print(pages[0].metadata)
    return pages

In [7]:
def ret_text_splitters(chunk_size,chunk_overlap,parent_chunk_size,child_chunk_size):
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    # This text splitter is used to create the parent documents
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=parent_chunk_size)
    # This text splitter is used to create the child documents. It should create documents smaller than the parent
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=child_chunk_size)
    # The vectorstore to use to index the child chunks
    
    return text_splitter, parent_splitter, child_splitter

In [18]:
# def main():
# Load pdf document and split in chunks
logging.info(f"Loading documents from {constants.SOURCE_DIR_FILE}")
pages = load_pdf(constants.SOURCE_DIR_FILE)
text_splitter, parent_splitter, child_splitter = ret_text_splitters(CHUNK_SIZE,
                                                                      CHUNK_OVERLAP, PARENT_CHUNK_SIZE, CHILD_CHUNK_SIZE)
texts = text_splitter.split_documents(pages)
parent_texts = parent_splitter.split_documents(pages)
child_texts = child_splitter.split_documents(pages)
logging.info(f"Split into {len(texts)} chunks of text")
logging.info(f"Split into {len(parent_texts)} chunks of text")
logging.info(f"Split into {len(child_texts)} chunks of text")
print(pages[4].page_content)

# Create embeddings
embeddings_instruct = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME)

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# Create in-memory and persist VectorDBs
db = Chroma.from_documents(
    texts,
    embeddings,
    persist_directory=PERSIST_DIRECTORY,
    client_settings=CHROMA_SETTINGS,
)

faiss_vector_db = FAISS.from_documents(texts, embedding=embeddings)
faiss_vector_db.save_local(f"{PERSIST_DIRECTORY}/faiss.parquet")
    

Loaded 50 of pages from the pdf file
{'source': '../SOURCE_DATA_DB/ConceptsofBiology-WEB_Chapter 1-2.pdf', 'file_path': '../SOURCE_DATA_DB/ConceptsofBiology-WEB_Chapter 1-2.pdf', 'page': 0, 'total_pages': 50, 'format': 'PDF 1.6', 'title': 'Concepts of Biology', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Prince 15 (www.princexml.com)', 'creationDate': 'D:20231205113534Z', 'modDate': 'D:20231205113534Z', 'trapped': ''}
FIGURE 1.6 A lot of energy is required for a California condor to fly. Chemical energy derived from food is used to power flight. California
condors are an endangered species; scientists have strived to place a wing tag on each bird to help them identify and locate each individual
bird. (credit: Pacific Southwest Region U.S. Fish and Wildlife)
Evolution
The diversity of life on Earth is a result of mutations, or random changes in hereditary material over time. These
mutations allow the possibility for organisms to adapt to a changing environme

In [ ]:
if __name__ == "__main__":
    main()